In [ ]:
import pathlib
import tempfile
import datetime
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.interpolate

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys._losslessjpeg
import pymedphys._wlutz.core
import pymedphys._wlutz.reporting
import pymedphys._wlutz.iview
import pymedphys._vendor.pylinac.winstonlutz

In [ ]:
results_dir = pathlib.Path(r'S:\Physics\Programming\results\Wlutz_Arc\2020-01-16_4299')

In [ ]:
image_paths = list(pathlib.Path(r'S:\Temp').glob('**/*.jpg'))
# images

In [ ]:
mechanical_icom_data_path = pathlib.Path(r'\\physics-server\iComLogFiles\mechanical\4299\20200116.csv')
icom_data = pd.read_csv(mechanical_icom_data_path)
icom_data

In [ ]:
diff = np.diff(icom_data['Monitor Units'])
diff[diff<0] = 0 

diff = diff[diff!=0]

plt.hist(diff, 100)

In [ ]:
initial_image_timestamps = np.array([
    datetime.datetime.fromtimestamp(os.path.getmtime(path))
    for path in image_paths
]).astype(np.datetime64)
# timestamps

In [ ]:
approximate_saving_time = np.timedelta64(1500, 'ms')

In [ ]:
time_diff = np.diff(initial_image_timestamps)
time_diff = np.concatenate([time_diff, [time_diff[-1]]])
image_adjusted_timestamps = initial_image_timestamps + approximate_saving_time

In [ ]:
icom_unique_timestamps, counts = np.unique(icom_data['Timestamp'], return_counts=True)
icom_unique_timestamps = icom_unique_timestamps.astype(np.datetime64)

In [ ]:
icom_adjusted_timestamps = []

for base_timestamp, count in zip(icom_unique_timestamps, counts):
    time_dt = 1000 / (count)
    
    current_dt = time_dt / 2
    for i in range(count):
        icom_adjusted_timestamps.append(base_timestamp + np.timedelta64(int(current_dt), 'ms'))
        current_dt += time_dt
        
        
icom_adjusted_timestamps = np.array(icom_adjusted_timestamps).astype('datetime64[ms]')

In [ ]:
# adjusted_timestamps = np.array(adjusted_timestamps).astype('datetime64[ms]').astype('uint64')

In [ ]:
def convert_timestamp(timestamp):
    return np.array(timestamp).astype('datetime64[ms]').astype('float64') / 60000

class Interpolator:
    def __init__(self, timestamps, data):
        interpolation = scipy.interpolate.UnivariateSpline(
            convert_timestamp(timestamps), data, k=1, s=0)
        self._interpolation = interpolation

    def __call__(self, timestamp):
        return self._interpolation(convert_timestamp(timestamp))

    def derivative(self, timestamp):
        dt = 1/60
        minutes = convert_timestamp(timestamp)
        return (self._interpolation(minutes + dt/2) - self._interpolation(minutes - dt/2)) / dt

keys = ['Gantry', 'Collimator', 'MLC distance at CRA', 'Jaw distance', 'Monitor Units']

interpolators = {
    key: Interpolator(icom_adjusted_timestamps, icom_data[key])
    for key in keys
}

In [ ]:
match_mask = np.abs(icom_adjusted_timestamps[:, None] - image_adjusted_timestamps[None, :]) < np.timedelta64(510, 'ms')

In [ ]:
icom_index, image_index = np.where(match_mask)

In [ ]:
_, unique_index = np.unique(image_index, return_index=True)
icom_index = icom_index[unique_index]
image_index = image_index[unique_index]

In [ ]:
energies = icom_data['Energy'][icom_index]

In [ ]:
valid_image_timestamps = np.array(image_adjusted_timestamps).astype(np.datetime64)[image_index]

In [ ]:
valid_image_paths = np.array(image_paths).astype(str)[image_index]

In [ ]:
def get_direction(timestamps):
    gantry_derivative = interpolators['Gantry'].derivative(valid_image_timestamps)
    result = np.ones_like(timestamps).astype(str)
    result[:] = ''
    result[gantry_derivative<-1] = 'counter-clockwise'
    result[gantry_derivative>1] = 'clockwise'
    
    return result

In [ ]:
image_values = {
    'Timestamp': valid_image_timestamps,
    'Path': valid_image_paths,
    'Energy': np.array(energies).astype(str),
    'Rough Dose Rate': interpolators['Monitor Units'].derivative(valid_image_timestamps),
    'Direction': get_direction(valid_image_timestamps),
    'Gantry': interpolators['Gantry'](valid_image_timestamps),
    'Collimator': interpolators['Collimator'](valid_image_timestamps),
    'Width': interpolators['MLC distance at CRA'](valid_image_timestamps) * 10,
    'Length': interpolators['Jaw distance'](valid_image_timestamps) * 10
}

per_image_data = pd.DataFrame(image_values)
per_image_data.to_csv(results_dir.joinpath('interpolated_iCom_results.csv'), index=False)

In [ ]:
image_results = {}

In [ ]:
bb_diameter = 8
penumbra = 2

# image_results = {}

for i, path in enumerate(image_paths):
    current_data = per_image_data.iloc[i]
    timestamp = current_data['Timestamp']
    
    try:
        image_results[timestamp]
        continue
    except KeyError:
        pass    
    
    print(path)
    loaded_image = pymedphys._losslessjpeg.imread(path)
    x, y, img = pymedphys._wlutz.iview.iview_image_transform(loaded_image)

    print(current_data)
    edge_lengths = [current_data['Width'], current_data['Length']]
    rotation = current_data['Collimator']
    

    try:
        bb_centre, field_centre, field_rotation = pymedphys._wlutz.core.find_field_and_bb(
            x, y, img, edge_lengths, bb_diameter, penumbra=penumbra, fixed_rotation=rotation, rounding=False, ignore_pylinac=True)
        
        image_results[timestamp] = {
            'BB Centre': bb_centre,
            'Field Centre': field_centre
        }

        pymedphys._wlutz.reporting.image_analysis_figure(
            x,
            y,
            img,
            bb_centre,
            field_centre,
            field_rotation,
            bb_diameter,
            edge_lengths,
            penumbra,
        )

        plt.show()
    except Exception as e:
        print(e)
        pass

In [ ]:
i = 50
loaded_image = pymedphys._losslessjpeg.imread(image_paths[i])
x, y, img = pymedphys._wlutz.iview.iview_image_transform(loaded_image)


In [ ]:
current_data = per_image_data.iloc[i]
current_data

In [ ]:
edge_lengths = [current_data['Width'], current_data['Length']]
edge_lengths

In [ ]:
bb_diameter = 8
penumbra = 2

In [ ]:
rotation = current_data['Collimator']
rotation

In [ ]:
bb_centre, field_centre, field_rotation = pymedphys._wlutz.core.find_field_and_bb(
    x, y, img, edge_lengths, bb_diameter, penumbra=penumbra, fixed_rotation=rotation, rounding=False, ignore_pylinac=True)

In [ ]:
pymedphys._wlutz.reporting.image_analysis_figure(
    x,
    y,
    img,
    bb_centre,
    field_centre,
    field_rotation,
    bb_diameter,
    edge_lengths,
    penumbra,
)